In [ ]:
import cv2
import numpy as np
from sort import Sort

In [ ]:
net = cv2.dnn.readNetFromDarknet("./yolov3.cfg", "./yolov3.weights")
classes = []
with open("./coco.names", "r") as f:
    classes = [line.strip() for line in f]

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
cap = cv2.VideoCapture('./Video1.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("./output_video.mp4", fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

CountCars = []
tracker = Sort()
count_car = 0

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    boxes = []
    confidences = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 2:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                left = int(center_x - width/2)
                top = int(center_y - height/2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    dets = []

    for i in indices:
        box = boxes[i[0]]
        width = box[2]
        height = box[3]
        left = box[0]
        top = box[1]
        cv2.rectangle(frame, (left, top), (left+width, top+height), (0, 255, 0), 2)
        dets.append([left, top, left+width, top+height, confidences[i]])

    dets = np.array(dets)

    if dets.shape[0] > 0:
        tracks = tracker.update(dets)
        for track in tracks:
            bbox = track.astype(np.int32)
            x1, y1, x2, y2, track_id = bbox
            if track_id not in CountCars:
                CountCars.append(track_id)
                count_car += 1

    out.write(frame)

In [ ]:
out.release()
cap.release()

print(f"Total count cars: {count_car}")